In [ ]:
# import libraries
import os
import re
import pandas as pd
import numpy as np
import regex
import pickle
import datetime
from datetime import datetime
import itertools

import nltk             
from nltk.corpus import stopwords
from nltk import word_tokenize

from nltk.metrics import ConfusionMatrix
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [ ]:
#%% load dataset properly
df10 = pd.read_excel('10samples.xlsx')

# count instances per classes
df10['set_label'].value_counts()

# set interested classes with integer labels 
df10.loc[df10['set_label'] == 'dansk', 'set_label'] = 1
df10.loc[df10['set_label'] == 'idræt', 'set_label'] = 2
df10.loc[df10['set_label'] == 'matematik', 'set_label'] = 3
df10.loc[df10['set_label'] == 'temporary', 'set_label'] = 4
df10.loc[df10['set_label'] == 'other', 'set_label'] = 0

# set all other classes as the class '0'
df10['set_label'] = np.where(
    (df10['set_label'] !=1) & (df10['set_label'] !=2) & (df10['set_label'] !=3) & (df10['set_label'] !=4), 0, df10['set_label'])

# class imbalance 
df10['set_label'].value_counts() 

In [ ]:
#%% data tuple 
tuple_10samples = (df10['name_activity'], df10['set_label']) #create dataframe tuple
set_10samples=[]
counter=0 #used for indexing
for _ in tuple_10samples[0]:
    set_10samples.append([(i.iloc[counter]) for i in tuple_10samples]) #append name emne and its label as a tuple
    counter+=1
print("10samples tuple length:", len(set_10samples))

word_features_all = pickle.load(open("all_features_noise.pickle", "rb"))

def document_features(document, word_features):
    
        document_words = set(document)
        features = {}
        for word in word_features :
            features['contains ({})'.format(word)] = (word in document_words)
        return features

#without features
data_to_test = [(document_features(word_tokenize(w.lower(), language='danish'),word_features_all), label) 
              for w, label in set_10samples]


In [ ]:
# the 2 classifiers NB LOG not cleaned
NB_noise = open("NB_notcleaned.pickle", "rb")
LOG_noise = open("LOG_notcleaned.pickle", "rb")
SVM_noise= open("SVM_notcleaned.pickle", "rb")

clf_nb = pickle.load(NB_noise)
clf_log = pickle.load(LOG_noise)
clf_svm = pickle.load(SVM_noise)

y_10 = [(y) for _, y in data_to_test]
target_names = ['0','1','2', '3', '4'] # classes

pred_nb = [clf_nb.classify(p) for p, _ in data_to_test]
pred_log = [clf_log.classify(p) for p, _ in data_to_test]
pred_svm = [clf_svm.classify(p) for p, _ in data_to_test]


f1_NB = f1_score(y_10, pred_nb, average='macro')
print("F1-Score NB:", round(f1_NB, 4),"\n")
print(classification_report(y_10, pred_nb, target_names=target_names))


f1_LOG = f1_score(y_10, pred_log, average='macro') 
print("F1-Score LOG:", round(f1_LOG, 4),"\n")
print(classification_report(y_10, pred_log, target_names=target_names))


f1_SVM = f1_score(y_10, pred_svm, average='macro') 
print("F1-Score SVM:", round(f1_SVM, 4),"\n")
print(classification_report(y_10, pred_svm, target_names=target_names))


#add results presdicitons in dataframe
df10['NB predictions']=pred_nb
df10['LOG predictions']=pred_log
df10['SVM predictions']=pred_svm
# =============================================================================
df10.to_excel('10samples_predictions_NB_LOG_SVM.xlsx')